In [1]:
import numpy as np
import config.config as config
from Environment import Env
from Agent_LSTM import *
from pathlib import Path

import sys; sys.path.append('../analysis/')
from my_utils import reset_seeds

In [2]:
def training(datapath, seed_number, Actor, Critic, 
             expnoise_std=0.8, TOTAL_EPISODE=5e4, value_noise_std=0):
    # get configures
    arg = config.ConfigGain(datapath)
    arg.SEED_NUMBER = seed_number
    arg.save()
    
    # reproducibility
    reset_seeds(arg.SEED_NUMBER)

    # initialize environment and agent
    env = Env(arg)
    agent = Agent(arg, Actor, Critic)
    agent.episodic = False
    agent.actor.value_noise_std = value_noise_std
    
    # define exploration noise
    noise = ActionNoise(arg.ACTION_DIM, mean=0, std=expnoise_std)
    
    # Loop now
    tot_t = 0
    episode = agent.initial_episode
    reward_log = []
    rewarded_trial_log = []
    step_log = []
    actor_loss_log = 0
    critic_loss_log = 0
    num_update = 1e-5
    dist_log = []

    LOG_FREQ = 100
    REPLAY_PERIOD = 4
    PRE_LEARN_PERIOD = arg.BATCH_SIZE * 50
    random_stop = True

    # Start loop
    while episode < TOTAL_EPISODE:
        # initialize a trial
        cross_start_threshold = False
        x = env.reset()
        agent.bstep.reset(env.pro_gains)
        
        last_action = torch.zeros(1, 1, arg.ACTION_DIM)
        last_action_raw = last_action.clone()

        state = torch.cat([x[-arg.OBS_DIM:].view(1, 1, -1), last_action,
                           env.target_position_obs.view(1, 1, -1), 
                           torch.zeros(1, 1, 1)], dim=2).to(arg.device)

        hiddenin = None
        tend = 0

        states = []
        actions = []
        rewards = []
        dones = []

        for t in range(arg.EPISODE_LEN):
            # 1. Check start threshold.
            if not cross_start_threshold and (last_action.abs() > arg.TERMINAL_ACTION).any():
                cross_start_threshold = True

            # 2. Take an action based on current state 
            # and previous hidden & cell states of LSTM units.
            action, action_raw, hiddenout = agent.select_action(state, hiddenin, action_noise=noise)
            if random_stop and np.random.rand() > 0.95:
                action = torch.zeros_like(action)

            # 3. Track next x in the environment.
            next_x, reached_target, relative_dist = env(x, action, t - tend)

            # 4. Next observation given next x.
            next_ox = agent.bstep(next_x)
            next_state = torch.cat([next_ox.view(1, 1, -1), action, env.target_position_obs.view(1, 1, -1),
                                    torch.ones(1, 1, 1) * t - tend + 1], dim=2).to(arg.device)

            # 5. Check whether stop.
            is_stop = env.is_stop(x, action)

            # 6. Give reward if stopped.          
            if is_stop and cross_start_threshold:
                reward = env.return_reward(x, reward_mode='mixed')
                done = torch.ones(1, 1, 1)
            else:
                reward = torch.zeros(1, 1, 1)
                done = torch.zeros(1, 1, 1)

            # 7. Append data.
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            dones.append(done)

            # 8. Update timestep.
            last_action_raw = action_raw
            last_action = action
            state = next_state
            x = next_x
            hiddenin = hiddenout
            tot_t += 1

            # 9. Update model.
            if len(agent.memory.memory) > PRE_LEARN_PERIOD and tot_t % REPLAY_PERIOD == 0:
                actor_loss, critic_loss = agent.learn()
                actor_loss_log += actor_loss
                critic_loss_log += critic_loss
                num_update += 1

            # 10. whether break.
            if is_stop and cross_start_threshold:
                step_log.append(t + 1 - tend)
                reward_log.append(reward.item())
                rewarded_trial_log.append(int(reached_target & is_stop))
                dist_log.append(relative_dist.item())
                # initialize a trial
                cross_start_threshold = False
                x = env.reset()
                agent.bstep.reset(env.pro_gains)
                state = torch.cat([x[-arg.OBS_DIM:].view(1, 1, -1), last_action,
                                   env.target_position_obs.view(1, 1, -1),
                                   torch.zeros(1, 1, 1)], dim=2).to(arg.device)
                tend = t + 1


        # store the last state
        states.append(state)
        # End of one trial, store trajectory into buffer.
        states = torch.cat(states)
        actions = torch.cat(actions).to(arg.device)
        rewards = torch.cat(rewards).to(arg.device)
        dones = torch.cat(dones).to(arg.device)
        agent.memory.push(states, actions, rewards, dones) 

        # store mirrored trajectories reflected along y-axis
        agent.memory.push(*agent.mirror_traj(states, actions), rewards, dones) 

        if episode % LOG_FREQ == LOG_FREQ - 1:
            # save
            agent.save(save_memory=False, episode=episode, pre_phase=False, full_param=True)
            
            print(f"t: {tot_t}, Ep: {episode}, action std: {noise.std:0.2f}")
            print(f"mean steps: {np.mean(step_log):0.3f}, "
                  f"mean reward: {np.mean(reward_log):0.3f}, "
                  f"reward rate: {np.sum(reward_log) / np.sum(step_log):0.3f}, "
                  f"rewarded fraction: {np.mean(rewarded_trial_log):0.3f}, "
                  f"relative distance: {np.mean(dist_log) * arg.LINEAR_SCALE:0.3f}, "
                  f"critic loss: {critic_loss_log / num_update:0.3f}, "
                  f"actor loss: {-actor_loss_log / (num_update/2):0.3f}")
            
            reward_rate = np.sum(reward_log) / np.sum(step_log)
            if reward_rate > 0.2:
                random_stop = False
                noise.reset(mean=0, std=0.5)
                
                
            #if np.mean(rewarded_trial_log) > 0.85:
            #    break

            reward_log = []
            rewarded_trial_log = []
            step_log = []
            actor_loss_log = 0
            critic_loss_log = 0
            num_update = 1e-5
            dist_log = []
            
        episode += 1

In [3]:
actors = ['Actor_novalue_learnb']
critics = ['Critic']
seeds = [[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
expnoise_std = 0.8
TOTAL_EPISODE = 1e4
folder_path = Path('D:/quitting_data/agents_no_curriculum')

value_noise_std = 0

In [4]:
for actor, critic, seed_ in zip(actors, critics, seeds):
    for seed in seed_:
        datapath = folder_path / f'{actor}{critic}' / f'seed{seed}' 
        exec(f'from {actor} import *'); exec(f'from {critic} import *')
        training(datapath, seed, Actor, Critic, expnoise_std, TOTAL_EPISODE, 
                 value_noise_std)

t: 10000, Ep: 99, action std: 0.80
mean steps: nan, mean reward: nan, reward rate: nan, rewarded fraction: nan, relative distance: nan, critic loss: 0.000, actor loss: 0.000


C:\Users\rz31\AppData\Local\anaconda3\envs\InductiveBias\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rz31\AppData\Local\anaconda3\envs\InductiveBias\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\rz31\AppData\Local\Temp\ipykernel_17932\2583562541.py:142: RuntimeWarning: invalid value encountered in double_scalars
  print(f"mean steps: {np.mean(step_log):0.3f}, "
C:\Users\rz31\AppData\Local\Temp\ipykernel_17932\2583562541.py:150: RuntimeWarning: invalid value encountered in double_scalars
  reward_rate = np.sum(reward_log) / np.sum(step_log)


t: 20000, Ep: 199, action std: 0.80
mean steps: nan, mean reward: nan, reward rate: nan, rewarded fraction: nan, relative distance: nan, critic loss: 0.000, actor loss: 0.000
t: 30000, Ep: 299, action std: 0.80
mean steps: nan, mean reward: nan, reward rate: nan, rewarded fraction: nan, relative distance: nan, critic loss: 0.000, actor loss: 0.000
t: 40000, Ep: 399, action std: 0.80
mean steps: nan, mean reward: nan, reward rate: nan, rewarded fraction: nan, relative distance: nan, critic loss: 0.000, actor loss: 0.000
t: 50000, Ep: 499, action std: 0.80
mean steps: 17.609, mean reward: 0.302, reward rate: 0.017, rewarded fraction: 0.018, relative distance: 263.212, critic loss: 0.022, actor loss: 0.005
t: 60000, Ep: 599, action std: 0.80
mean steps: 17.509, mean reward: 0.365, reward rate: 0.021, rewarded fraction: 0.026, relative distance: 253.123, critic loss: 0.052, actor loss: -0.009
t: 70000, Ep: 699, action std: 0.80
mean steps: 16.357, mean reward: 0.452, reward rate: 0.028, re

t: 460000, Ep: 4599, action std: 0.80
mean steps: 16.029, mean reward: 0.045, reward rate: 0.003, rewarded fraction: 0.002, relative distance: 303.478, critic loss: 0.004, actor loss: -0.056
t: 470000, Ep: 4699, action std: 0.80
mean steps: 17.548, mean reward: 0.021, reward rate: 0.001, rewarded fraction: 0.000, relative distance: 308.248, critic loss: 0.003, actor loss: -0.058
t: 480000, Ep: 4799, action std: 0.80
mean steps: 17.057, mean reward: 0.073, reward rate: 0.004, rewarded fraction: 0.004, relative distance: 303.360, critic loss: 0.003, actor loss: -0.058
t: 490000, Ep: 4899, action std: 0.80
mean steps: 16.018, mean reward: 0.025, reward rate: 0.002, rewarded fraction: 0.000, relative distance: 310.639, critic loss: 0.005, actor loss: -0.057
t: 500000, Ep: 4999, action std: 0.80
mean steps: 16.446, mean reward: 0.021, reward rate: 0.001, rewarded fraction: 0.000, relative distance: 307.887, critic loss: 0.004, actor loss: -0.057
t: 510000, Ep: 5099, action std: 0.80
mean st

t: 890000, Ep: 8899, action std: 0.80
mean steps: 5.691, mean reward: 0.740, reward rate: 0.130, rewarded fraction: 0.060, relative distance: 240.826, critic loss: 0.475, actor loss: 2.514
t: 900000, Ep: 8999, action std: 0.80
mean steps: 5.900, mean reward: 1.163, reward rate: 0.197, rewarded fraction: 0.104, relative distance: 240.325, critic loss: 0.420, actor loss: 3.354
t: 910000, Ep: 9099, action std: 0.80
mean steps: 5.978, mean reward: 1.112, reward rate: 0.186, rewarded fraction: 0.098, relative distance: 236.775, critic loss: 0.521, actor loss: 4.308
t: 920000, Ep: 9199, action std: 0.80
mean steps: 6.314, mean reward: 1.438, reward rate: 0.228, rewarded fraction: 0.129, relative distance: 226.516, critic loss: 0.552, actor loss: 5.368
t: 930000, Ep: 9299, action std: 0.50
mean steps: 7.536, mean reward: 2.789, reward rate: 0.370, rewarded fraction: 0.262, relative distance: 199.780, critic loss: 0.577, actor loss: 6.474
t: 940000, Ep: 9399, action std: 0.50
mean steps: 7.367

...

t: 180000, Ep: 1799, action std: 0.80
mean steps: 16.057, mean reward: 0.533, reward rate: 0.033, rewarded fraction: 0.041, relative distance: 250.370, critic loss: 0.056, actor loss: -0.036
t: 190000, Ep: 1899, action std: 0.80
mean steps: 16.300, mean reward: 0.521, reward rate: 0.032, rewarded fraction: 0.038, relative distance: 242.686, critic loss: 0.055, actor loss: -0.054
t: 200000, Ep: 1999, action std: 0.80
mean steps: 16.124, mean reward: 0.401, reward rate: 0.025, rewarded fraction: 0.027, relative distance: 248.370, critic loss: 0.052, actor loss: -0.067
t: 210000, Ep: 2099, action std: 0.80
mean steps: 15.083, mean reward: 0.383, reward rate: 0.025, rewarded fraction: 0.024, relative distance: 227.704, critic loss: 0.052, actor loss: -0.074
t: 220000, Ep: 2199, action std: 0.80
mean steps: 16.128, mean reward: 0.180, reward rate: 0.011, rewarded fraction: 0.010, relative distance: 255.460, critic loss: 0.048, actor loss: -0.105
t: 230000, Ep: 2299, action std: 0.80
mean st

t: 620000, Ep: 6199, action std: 0.50
mean steps: 13.102, mean reward: 6.395, reward rate: 0.488, rewarded fraction: 0.612, relative distance: 115.283, critic loss: 0.982, actor loss: 24.579
t: 630000, Ep: 6299, action std: 0.50
mean steps: 12.942, mean reward: 6.243, reward rate: 0.482, rewarded fraction: 0.600, relative distance: 118.955, critic loss: 1.004, actor loss: 24.652
t: 640000, Ep: 6399, action std: 0.50
mean steps: 12.779, mean reward: 6.630, reward rate: 0.519, rewarded fraction: 0.639, relative distance: 112.054, critic loss: 1.004, actor loss: 24.696
t: 650000, Ep: 6499, action std: 0.50
mean steps: 12.482, mean reward: 6.245, reward rate: 0.500, rewarded fraction: 0.604, relative distance: 124.095, critic loss: 0.990, actor loss: 24.718
t: 660000, Ep: 6599, action std: 0.50
mean steps: 13.246, mean reward: 6.563, reward rate: 0.496, rewarded fraction: 0.634, relative distance: 112.139, critic loss: 0.985, actor loss: 24.748
t: 670000, Ep: 6699, action std: 0.50
mean st

t: 60000, Ep: 599, action std: 0.80
mean steps: 12.900, mean reward: 0.353, reward rate: 0.027, rewarded fraction: 0.025, relative distance: 247.205, critic loss: 0.086, actor loss: 0.595
t: 70000, Ep: 699, action std: 0.80
mean steps: 16.998, mean reward: 0.339, reward rate: 0.020, rewarded fraction: 0.020, relative distance: 256.300, critic loss: 0.093, actor loss: 0.823
t: 80000, Ep: 799, action std: 0.80
mean steps: 17.065, mean reward: 0.300, reward rate: 0.018, rewarded fraction: 0.020, relative distance: 264.610, critic loss: 0.091, actor loss: 0.560
t: 90000, Ep: 899, action std: 0.80
mean steps: 17.658, mean reward: 0.300, reward rate: 0.017, rewarded fraction: 0.019, relative distance: 252.168, critic loss: 0.076, actor loss: 0.369
t: 100000, Ep: 999, action std: 0.80
mean steps: 17.199, mean reward: 0.425, reward rate: 0.025, rewarded fraction: 0.030, relative distance: 255.590, critic loss: 0.065, actor loss: 0.252
t: 110000, Ep: 1099, action std: 0.80
mean steps: 16.403, m

t: 500000, Ep: 4999, action std: 0.50
mean steps: 12.757, mean reward: 6.124, reward rate: 0.480, rewarded fraction: 0.589, relative distance: 123.717, critic loss: 1.123, actor loss: 23.272
t: 510000, Ep: 5099, action std: 0.50
mean steps: 12.723, mean reward: 6.482, reward rate: 0.509, rewarded fraction: 0.630, relative distance: 119.648, critic loss: 1.143, actor loss: 23.348
t: 520000, Ep: 5199, action std: 0.50
mean steps: 12.897, mean reward: 6.609, reward rate: 0.512, rewarded fraction: 0.643, relative distance: 117.146, critic loss: 1.121, actor loss: 23.439
t: 530000, Ep: 5299, action std: 0.50
mean steps: 12.939, mean reward: 6.414, reward rate: 0.496, rewarded fraction: 0.615, relative distance: 115.582, critic loss: 1.130, actor loss: 23.491
t: 540000, Ep: 5399, action std: 0.50
mean steps: 13.106, mean reward: 6.462, reward rate: 0.493, rewarded fraction: 0.624, relative distance: 116.846, critic loss: 1.138, actor loss: 23.589
t: 550000, Ep: 5499, action std: 0.50
mean st

t: 930000, Ep: 9299, action std: 0.50
mean steps: 12.658, mean reward: 6.291, reward rate: 0.497, rewarded fraction: 0.611, relative distance: 124.347, critic loss: 1.082, actor loss: 24.668
t: 940000, Ep: 9399, action std: 0.50
mean steps: 12.540, mean reward: 6.333, reward rate: 0.505, rewarded fraction: 0.616, relative distance: 122.447, critic loss: 1.094, actor loss: 24.688
t: 950000, Ep: 9499, action std: 0.50
mean steps: 12.364, mean reward: 6.257, reward rate: 0.506, rewarded fraction: 0.609, relative distance: 125.097, critic loss: 1.116, actor loss: 24.666
t: 960000, Ep: 9599, action std: 0.50
mean steps: 12.470, mean reward: 6.326, reward rate: 0.507, rewarded fraction: 0.612, relative distance: 121.248, critic loss: 1.082, actor loss: 24.675
t: 970000, Ep: 9699, action std: 0.50
mean steps: 12.605, mean reward: 6.539, reward rate: 0.519, rewarded fraction: 0.636, relative distance: 118.994, critic loss: 1.068, actor loss: 24.703
t: 980000, Ep: 9799, action std: 0.50
mean st